In [ ]:
import importlib
import pc_e

from datamodules import EMNIST as MyDataModule
from get_arch import get_architecture
from custom_callbacks import ErrorConvergenceCallback

from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger

In [ ]:
importlib.reload(pc_e)
from pc_e import PCE


# 0: load dataset as Lightning DataModule

batch_size = 64
datamodule = MyDataModule(batch_size)
print("Training on", datamodule.dataset_name)

# 1: Set up Lightning trainer
# logger = False
logger = WandbLogger(project="XXX", entity="XXX", mode="online")

trainer = Trainer(
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[ErrorConvergenceCallback()],
    max_epochs=8,
    inference_mode=False,  # inference_mode would interfere with the state backward pass
    limit_predict_batches=1,  # enable 1-batch prediction
)

# 2: Get architecture that belongs to this dataset
architecture = get_architecture(dataset=datamodule.dataset_name)

# 3: Initiate model and train it
pc = PCE(architecture, iters=16, e_lr=0.1, w_lr=0.001)
trainer.fit(pc, datamodule=datamodule)

In [ ]:
trainer.test(pc, datamodule=datamodule)

In [ ]:
# pc.predict_targets = {"color", "shape", "angle"}
# pc.predict_targets = {"img"}
pc.predict_targets = {"y"}
trainer.callbacks.append(datamodule.prediction_callback())
trainer.predict(pc, datamodule=datamodule)
trainer.callbacks = trainer.callbacks[:-1]